Generacion de pedidos y detalle

Configuracion y conexion


In [19]:
import pandas as pd
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
import time

DATABASE_URL = "tu_url_de_neon_aqui"
engine = create_engine('Url de neon')

try:
    df_usuarios = pd.read_sql('SELECT "idUsuario", "segmentoCliente" FROM "Usuario"', engine)
    df_productos = pd.read_sql('SELECT "idProducto", "precio", "idCategoria" FROM "producto"', engine)
    print(f" Datos base cargados: {len(df_usuarios)} usuarios y {len(df_productos)} productos.")
except Exception as e:
    print(f" Error al leer datos iniciales: {e}")
    df_usuarios = pd.DataFrame()

ArgumentError: Could not parse SQLAlchemy URL from given URL string

# Logica de simulacion

In [16]:
mapeo_gustos = {
    'CatLover': 2,
    'DogLover': 1,
    'Aves': 6,
    'Exóticos': 6
}

metodos_pago = ['Tarjeta de Crédito', 'Mercado Pago', 'Transferencia', 'Efectivo']

def obtener_productos_por_segmento(segmento):
    id_cat = mapeo_gustos.get(segmento, 2)
    candidatos = df_productos[df_productos['idCategoria'] == id_cat]
    return candidatos if not candidatos.empty else df_productos

# Carga de base de datos

In [18]:
total_pedidos_creados = 0

for index, usuario in df_usuarios.iterrows():
    cantidad_pedidos_usuario = random.randint(3, 8)

    print(f"procesando Usuario ID {usuario['idUsuario']} ({usuario['segmentoCliente']}): Generando {cantidad_pedidos_usuario} pedidos...")

    for i in range(cantidad_pedidos_usuario):
        try:
            with engine.begin() as conn:

                productos_validos = obtener_productos_por_segmento(usuario['segmentoCliente'])
                metodo = random.choice(metodos_pago)

                dias_atras = random.randint(1, 60)
                fecha = datetime.now() - timedelta(days=dias_atras, hours=random.randint(0,23), minutes=random.randint(0,59))


                query_p = text("""
                    INSERT INTO "Pedido" (fecha, total, "metodoPago", "idUsuario")
                    VALUES (:fecha, 0, :metodo, :id_u) RETURNING "idPedido"
                """)

                res = conn.execute(query_p, {
                    "fecha": fecha,
                    "metodo": metodo,
                    "id_u": int(usuario['idUsuario'])
                })
                id_pedido = res.fetchone()[0]

               #detalle
                total_venta = 0
                n_items = random.randint(1, 4)
                items_a_comprar = productos_validos.sample(n=min(n_items, len(productos_validos)))

                for _, prod in items_a_comprar.iterrows():
                    cant = random.randint(1, 3)
                    precio_py = float(prod['precio'])
                    id_prod_py = int(prod['idProducto'])
                    subtotal = cant * precio_py
                    total_venta += subtotal
                    query_d = text("""
                        INSERT INTO "Detalle" (cantidad, "precioUnitario", "idPedido", "idProducto")
                        VALUES (:cant, :precio, :id_p, :id_prod)
                    """)

                    conn.execute(query_d, {
                        "cant": int(cant),
                        "precio": precio_py,
                        "id_p": int(id_pedido),
                        "id_prod": id_prod_py
                    })

                conn.execute(text('UPDATE "Pedido" SET total = :t WHERE "idPedido" = :id'),
                             {"t": total_venta, "id": id_pedido})

                total_pedidos_creados += 1

            time.sleep(0.05)

        except OperationalError:
            print(f" Parpadeo de red. Saltando pedido {i+1}...")
        except Exception as e:
            print(f" Error inesperado en pedido: {e}")

print(f" fin... Se crearon un total de {total_pedidos_creados} pedidos nuevos.")

procesando Usuario ID 1 (DogLover): Generando 5 pedidos...
procesando Usuario ID 2 (Aves): Generando 6 pedidos...
procesando Usuario ID 3 (CatLover): Generando 3 pedidos...
procesando Usuario ID 4 (Aves): Generando 5 pedidos...
procesando Usuario ID 5 (Aves): Generando 6 pedidos...
procesando Usuario ID 6 (Exóticos): Generando 5 pedidos...
procesando Usuario ID 7 (Exóticos): Generando 8 pedidos...
procesando Usuario ID 8 (DogLover): Generando 4 pedidos...
procesando Usuario ID 9 (Aves): Generando 3 pedidos...
procesando Usuario ID 10 (Exóticos): Generando 6 pedidos...
procesando Usuario ID 11 (Exóticos): Generando 8 pedidos...
procesando Usuario ID 12 (Exóticos): Generando 6 pedidos...
procesando Usuario ID 13 (Aves): Generando 3 pedidos...
procesando Usuario ID 14 (Aves): Generando 6 pedidos...
procesando Usuario ID 15 (CatLover): Generando 3 pedidos...
procesando Usuario ID 16 (CatLover): Generando 8 pedidos...
procesando Usuario ID 17 (CatLover): Generando 3 pedidos...
procesando Us